In [11]:
import sys
from pathlib import Path

proj_root = Path.cwd().parent        # .../urban_taxonomy
src = proj_root / "src"              # .../urban_taxonomy/src
if str(src) not in sys.path:
    sys.path.insert(0, str(src))

import importlib.util
print("core spec:", importlib.util.find_spec("core"))

core spec: ModuleSpec(name='core', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000001CDF21C0230>, origin='d:\\Work\\Github_Morphotopes\\urban_taxonomy\\src\\core\\__init__.py', submodule_search_locations=['d:\\Work\\Github_Morphotopes\\urban_taxonomy\\src\\core'])


In [12]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve() / "src"))  # project root / src
from core.generate_streets import read_overture_region_streets

import gc
import glob

import geopandas as gpd
import momepy as mm
import numpy as np
import pandas as pd
import shapely
from libpysal.graph import Graph, read_parquet
import datetime



street_output_dir = 'D:/Work/Github_Morphotopes/data/overture_streets/'
input_datadir = "D:/Work/Github_Morphotopes/data/"

Reading streets…


FileNotFoundError: [WinError 2] Failed to open local file 'D:/Work/Github_Morphotopes/data/overture_streets/streets_0.parquet'. Detail: [Windows error 2] The system cannot find the file specified.


In [ ]:
regions_v = "0"

## Run sequentially for all streets

##### Reads cadaster hull

In [ ]:
region_hulls = gpd.read_parquet(input_datadir + f"cadastre_regions_hull.parquet")
region_hulls.shape

(1, 1)

In [ ]:
region_hulls

,convex_hull
labels,
0,"POLYGON ((393200 5707900, 393200 5709300, 3933..."


## Run in parallel for all non-procesed streets in region_hulls

##### Define a worker function


In [ ]:
def process_and_save(region_hull, region_id):
    print('Processing', region_id, datetime.datetime.now())
    streets = read_overture_region_streets(region_hull, region_id)
    streets.to_parquet(street_output_dir + f'streets_{region_id}.parquet')

In [ ]:
downloaded_streets = glob.glob(street_output_dir + '*.parquet')
downloaded_streets = [int(s.split('_')[-1][:-3]) for s in downloaded_streets]

In [ ]:
downloaded_streets

[]

In [ ]:
region_hulls = region_hulls.loc[~region_hulls.index.isin(downloaded_streets), ]
region_hulls

,convex_hull
labels,
0,"POLYGON ((393200 5707900, 393200 5709300, 3933..."


##### after following code is run, streets_0.parquet is created in Overture_street folder


In [ ]:
%%capture cap

from joblib import Parallel, delayed

n_jobs = -1
new = Parallel(n_jobs=n_jobs)(
    delayed(process_and_save)(region_hull.iloc[0], region_id) for region_id, region_hull in region_hulls.to_crs(epsg=4326).iterrows()
)


In [ ]:
import geopandas as gpd

gdf = gpd.read_parquet(street_output_dir + "/streets_0.parquet")
print(gdf.head())       # first 5 rows
print(gdf.columns)      # list of columns
print(gdf.crs) 

                                   id  \
80   0871f12cd0ffffff046fbde43ca9c48a   
91   0891f12cd083ffff047ffcf6a6b7f443   
92   0891f12cd083ffff046fbef03bf6a352   
112  08a1f12cd0807fff047fbcf07e483371   
113  08a1f12cd0807fff046fbd039b62a421   

                                              geometry  \
80   LINESTRING (7.46232 51.51425, 7.46227 51.51376...   
91   LINESTRING (7.46598 51.51293, 7.46535 51.51294...   
92   LINESTRING (7.46534 51.51373, 7.46534 51.51346...   
112  LINESTRING (7.46533 51.51425, 7.46533 51.5142,...   
113  LINESTRING (7.46582 51.51426, 7.46581 51.51422...   

                                                  bbox  version  \
80   {'xmax': 7.4690022468566895, 'xmin': 7.4621915...        0   
91   {'xmax': 7.465977191925049, 'xmin': 7.46504640...        0   
92   {'xmax': 7.465339660644531, 'xmin': 7.46529340...        0   
112  {'xmax': 7.465331554412842, 'xmin': 7.46532964...        0   
113  {'xmax': 7.465828895568848, 'xmin': 7.46581315...        0   

 

#### give CRS to streets and save as geopackage

In [ ]:
gdf = gpd.read_parquet(street_output_dir + f"streets_0.parquet")

In [ ]:
print("CRS:", gdf.crs)
print("Rows:", len(gdf))
print("Bounds (xmin, ymin, xmax, ymax):", gdf.total_bounds)

CRS: None
Rows: 1082
Bounds (xmin, ymin, xmax, ymax): [ 7.453397  51.5016042  7.5446298 51.5815978]


In [ ]:
gdf = gdf.set_crs(4326) 

In [ ]:
m = gdf.explore(
    basemap="CartoDB positron",
    color="red",     # street color
    tooltip=True     # show attributes on hover
)
m.save("map_basemap.html")

In [ ]:
out_path = "D:\Work\Github_Morphotopes\data\streets.gpkg"
gdf.to_file(street_output_dir + "streets.gpkg" , driver="GPKG")

<>:1: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\W'
C:\Users\Itrat\AppData\Local\Temp\ipykernel_22928\3857955552.py:1: SyntaxWarning: invalid escape sequence '\W'
  out_path = "D:\Work\Github_Morphotopes\data\streets.gpkg"


In [ ]:
cap.stderr

''